# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mananara,-16.1667,49.7667,22.75,92,29,3.26,MG,1714958558
1,1,puerto ayora,-0.7393,-90.3518,23.64,96,97,2.24,EC,1714958559
2,2,jamestown,42.0970,-79.2353,17.23,96,100,2.57,US,1714958420
3,3,yellowknife,62.4560,-114.3525,14.67,54,20,5.14,CA,1714958413
4,4,tura,25.5198,90.2201,22.51,83,63,0.90,IN,1714958559


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City'    
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Cloudiness'] == 0) & 
                                (city_data_df['Wind Speed'] < 5) & 
                                (city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Max Temp'] < 27)]

# Drop any rows with null values
ideal_weather_df.dropna(inplace = True)

# Display sample data
ideal_weather_df

C:\Users\andre\AppData\Local\Temp\ipykernel_30368\1111610287.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,praia,14.9215,-23.5087,25.30,33,0,2.57,CV,1714958266
74,74,peruibe,-24.3200,-46.9983,21.86,82,0,1.55,BR,1714958573
103,103,maua,-23.6678,-46.4614,25.20,65,0,1.93,BR,1714958579
113,113,metlili chaamba,32.2667,3.6333,25.52,17,0,4.63,DZ,1714958580
124,124,comandante fontana,-25.3345,-59.6821,22.43,78,0,2.59,AR,1714958583
209,209,barentu,15.1139,37.5928,25.73,16,0,2.14,ER,1714958600
217,217,minab,27.1467,57.0801,23.18,73,0,0.00,IR,1714958602
226,226,hamilton,39.1834,-84.5333,21.60,78,0,0.00,US,1714958604
240,240,tindouf,27.6711,-8.1474,24.25,20,0,3.60,DZ,1714958606
259,259,santa luzia,-19.7697,-43.8514,22.50,75,0,3.60,BR,1714958610


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,praia,CV,14.9215,-23.5087,33,
74,peruibe,BR,-24.3200,-46.9983,82,
103,maua,BR,-23.6678,-46.4614,65,
113,metlili chaamba,DZ,32.2667,3.6333,17,
124,comandante fontana,AR,-25.3345,-59.6821,78,
209,barentu,ER,15.1139,37.5928,16,
217,minab,IR,27.1467,57.0801,73,
226,hamilton,US,39.1834,-84.5333,78,
240,tindouf,DZ,27.6711,-8.1474,20,
259,santa luzia,BR,-19.7697,-43.8514,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
praia - nearest hotel: Casa Sodadi
peruibe - nearest hotel: Brisamar
maua - nearest hotel: Go Inn Santo André
metlili chaamba - nearest hotel: No hotel found
comandante fontana - nearest hotel: No hotel found
barentu - nearest hotel: No hotel found
minab - nearest hotel: هتل صدف
hamilton - nearest hotel: North Vista Manor
tindouf - nearest hotel: محمد بوسبي
santa luzia - nearest hotel: Alcoboca
lamar - nearest hotel: 1st St Lofts
toliara - nearest hotel: Ambary
tokar - nearest hotel: No hotel found
tsuruoka - nearest hotel: 長寿温泉
kazerun - nearest hotel: هتل بیشاپور
karbala - nearest hotel: فندق القمر
antonio enes - nearest hotel: Hotel Quirimbas
piuma - nearest hotel: Pousada Tropical
panama city beach - nearest hotel: Shores of Panama
childress - nearest hotel: Red Roof Inn Childress


,City,Country,Lat,Lng,Humidity,Hotel Name
52,praia,CV,14.9215,-23.5087,33,Casa Sodadi
74,peruibe,BR,-24.3200,-46.9983,82,Brisamar
103,maua,BR,-23.6678,-46.4614,65,Go Inn Santo André
113,metlili chaamba,DZ,32.2667,3.6333,17,No hotel found
124,comandante fontana,AR,-25.3345,-59.6821,78,No hotel found
209,barentu,ER,15.1139,37.5928,16,No hotel found
217,minab,IR,27.1467,57.0801,73,هتل صدف
226,hamilton,US,39.1834,-84.5333,78,North Vista Manor
240,tindouf,DZ,27.6711,-8.1474,20,محمد بوسبي
259,santa luzia,BR,-19.7697,-43.8514,75,Alcoboca


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name','Country']    
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)